# An experiment/simulation to understand Simpson Paradox

## Motivation

Thông qua một ví dụ về thử nghiệm ngẫu nhiên có kiểm soát (RCT hay A/B test) trong một chiến dịch Marketing, ta sẽ thấy được:

- Sự khác biệt giữa quan sát ngẫu nhiên trong quần thể (population) và khi có sự xuất hiện của 1 RCT;
- Hạn chế của thống kê quy ước trong cả hai trường hợp -> tầm quan trọng của suy luận nhân quả.



In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
sns.set_palette("Set1")

## Scenario 1: Random observed data

Marketer có trong tay 1 `DataFrame` gồm 2 biến binary:

- `T`: 0 ~ không thực hiện chiến dịch quảng cáo mới, 1 ~ có thực hiện
- `Y`: 0 ~ không mua hàng trong session đầu ngay sau chiến dịch, 1 ~ có mua (hoặc có click vào sản phẩm ~ CTR).

In [ ]:
def simulate_bin(
    sample_size: int = 1000,
    force_treatment: np.ndarray = None,
    obs_confouder: bool = False,
) -> pd.DataFrame:
    """
    Simulate a binary outcome Y from a binary treatment T and a binary confounder C.
    All are binaries

    Causal Diagram:
    Nodes: (C, T, Y)
    Edges: (C -> T, C -> Y, T-> Y)

    Args:
    sample_size: size of the sample;
    force_treatment: treatment or not;
    obs_confounder: to show/observe confounder or not;

    Return:
    A 2-columns pandas dataframe if obs_confounder = False,
    A 3-columns pandas dataframe if obs_confounder = True
    """
    p_c = 0.5  # confounder prob
    p_t_c = [0.8, 0.2]  # treatment and control portion
    p_y_tc = [0.1, 0.3, 0.7, 0.9]

    # confounder
    c = np.random.binomial(n=1, p=p_c, size=sample_size)

    if force_treatment is not None:
        assert len(force_treatment) == sample_size
        t = force_treatment
    else:
        p_t = np.choose(c, p_t_c)
        # https://numpy.org/doc/2.1/reference/generated/numpy.choose.html
        t = np.random.binomial(n=1, p=p_t, size=sample_size)

    p_y = np.choose(c * 2 + t, p_y_tc)
    y = np.random.binomial(n=1, p=p_y, size=sample_size)

    if obs_confouder:
        return pd.DataFrame({"T": t, "Y": y, "C": c})

    return pd.DataFrame({"T": t, "Y": y})

Simulate a sample of 1000, no treatment, not show confounder:

In [ ]:
obs_data = simulate_bin(sample_size=1000, force_treatment=None, obs_confouder=False)
obs_data.head()

,T,Y
0,1,1
1,0,1
2,0,1
3,0,0
4,1,0


### Traditional statistics

A cross tabulated report to view data:

In [5]:
xtab = pd.crosstab(obs_data['T'], obs_data['Y'])

xtab

Y,0,1
T,,
0,229,268
1,296,207


In [7]:
100 * xtab[1] / xtab.aggregate('sum', axis=1)

T
0    53.923541
1    41.153082
dtype: float64

In [8]:
41.153082 - 53.923541

-12.770459000000002

In [17]:
def estimate_effect(ds):
    
    base = ds[ds['T'] == 0]
    variant = ds[ds['T'] == 1]
    
    delta = variant.Y.mean() - base.Y.mean()
    
    delta_err = 1.96 * np.sqrt(
        variant.Y.var() / variant.shape[0] +
        base.Y.var() / base.shape[0]
    )
    
    return f"Avg effect: {100*delta:.2f}% : {100*(delta - delta_err):.2f}% to {100*(delta + delta_err):.2f}%"

estimate_effect(obs_data)

'Avg effect: -12.77% : -18.92% to -6.62%'

## Scenario 2: RCT

## Explanation

## Backdoor adjustment with `DoWhy`

## Conclusion

1. Dữ liệu bản thân nó không quan trọng bằng **quá trình/cơ chế** sinh dữ liệu;
2. Khi thực hiện RCT, cần chú ý tới **confounders**. Đồng thời RCT cũng sẽ ảnh hưởng trực tiếp đến tính ngẫu nhiên của dữ liệu (ta đã thay đổi quá khứ);
3. **Association/Correlation** khác/không tương đương với **Causal relationship**;
4. Simpson paradox là một trường hợp cực đoan khi RCT không kiểm soát tốt confounders, vi phạm giả định về **tính độc lập có điều kiện** của can thiệp (treatment);
5. Nếu quy trình phân tích không được dẫn đường bởi quy luật nhân quả, rất dễ rơi vào bẫy nghịch lý, dẫn đến **quyết định sai lầm**.

## References

- BS Le Ngoc Kha Nhi: <https://www.lengockhanhi.com/post/ngh%E1%BB%8Bch-l%C3%BD-simpson>